In [2]:
import pandas as pd
import numpy as np
import dash 
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
from datetime import datetime as dt

In [3]:
app = dash.Dash()

In [4]:
df2017 = pd.read_excel(r'C:\Users\ykkow\.spyder-py3\2017_test.xlsx')
df2018 = pd.read_excel(r'C:\Users\ykkow\.spyder-py3\2018_test.xlsx')
df2019 = pd.read_excel(r'C:\Users\ykkow\.spyder-py3\2019_test.xlsx')
df2017=df2017[[ 'ArtikelCode','ArtikelName', 'Quantity', 'Sales', 'StoreCode', 'Store','DateTime', 'BigClass', 'MiddleClass', 'FineClass']]
df2018=df2018[[ 'ArtikelCode','ArtikelName', 'Quantity', 'Sales', 'StoreCode', 'Store','DateTime', 'BigClass', 'MiddleClass', 'FineClass']]
df=pd.concat([df2017,df2018,df2019],join='outer',ignore_index=True,sort=False)
df=df[df['Store'].isin(['Alexa','GB', 'MOB','Schloss', 'WA'])]  #剔出berlin125和milano的数据
df['DateTime']= df['DateTime'].apply(lambda x: dt.strptime(x,'%Y-%m-%d %H:%M:%S'))
df['Year']=df['DateTime'].apply(lambda x: x.year)
df['Month']=df['DateTime'].apply(lambda x: x.month)
df['Day']=df['DateTime'].apply(lambda x: x.day)

In [5]:
df_table=df[['Store', 'BigClass', 'FineClass', 'ArtikelCode','Quantity','Sales', 'Year', 'Month','Day','DateTime']]

In [6]:
app.layout=html.Div([
        html.Div([
                html.H2('Lings Dash'),
                html.H6('A Simply Sales Analysis Dashboard'),
                html.H6('(Note: All the data in this Dashboard is fake and made up.)')
                ],style={'textAlign':'center'}
                ),
        html.Div(
                [html.P('Please choose one feature:'),
                dcc.Dropdown(
                        id='dropdown',
                        options=[
                                {'label':'Year', 'value':'Year'},
                                {'label':'Store', 'value':'Store'},
                                {'label':'BigClass', 'value':'BigClass'},
                                {'label':'FineClass', 'value':'FineClass'}
                                ],
                        value='Store',
                        multi=False,
                        clearable=False,
                        style={'color':'black','fontWeight':'bold','border':'1px solid blue'})
                ],
                ),
        html.Br(),
        html.Div([
                dash_table.DataTable(
                        id='tabledata',
                        columns=[
                                {'name': i, 'id':i,'deletable':True,'selectabel':True } for i in df_table.columns
                                ],
                        data=df.to_dict('records'),  
                        #to_dict('record'):将df转换成列表字典，列表里元素以一行的数据为一个字典，字典里以列名为key，列值weivalue[{'col1': v_11,'col2':v_12},{'col1':v_21,'col2':v_22},{'col1':v_31,'col2':v_32}]
                        #columns
                        sort_action='native',
                        sort_mode='multi',
                        column_selectable='multi',
                        selected_columns='multi',
                        filter_action='native',
                        #fixed_columns={'header':True,'data':1}, 
                        #page 
                        page_action='native',
                        page_size=10,
                        #rows
                        selected_rows=[],
                        fixed_rows={'headers':True},  #将表头固定
                        #derived_virtual_indices=[],
                        row_deletable=True,
                        row_selectable='multi',
                        #style_cell={    #ensure adequate header width when text is shorter than cell's text
                        #   'minWidth':95, 'maxWidth':95, 'width':95     
                        #        },
                        
                        #style
                        style_as_list_view=True,
                        style_table={'width':'100%','height':'300px','overflowX':'auto'}, #设置垂直滚动条
                        style_cell={'padding':'5px'},
                        style_cell_conditional=[
                                {
                                    'if':{'column_id':c},
                                    'textAlign': 'left'
                                    } for c in ['Store', 'BigClass', 'FineClass']
                                ],
                        style_data={    # overflow cells' content into multi lines
                                
                                'overflow':'hidden', 'textOverflow':'ellipses','maxWidth':0, #将长文字用...省略
                                },
                        #
                        style_data_conditional=[
                                {'if': {'row_index':'odd'},
                                'backgroundColor':'rgb(248,248,248)'
                                },              #将偶数行设置为灰色
                                {'if':{'column_id':'FineClass',
                                       'filter_query':'{FineClass} eq "Woody Toys"'},
                                'backgroundColor':'green',
                                'color':'white', 'fontWeight':'bold'
                                
                                },      #将sales列大于50的背景为绿色
                                ],
                        #将表头背景调为黑底白字
                        style_header={'fontWeight':'bold',
                                      'backgroundColor':'black', 'color':'white'}
                        )
                
                ]
                ),
        html.Br(),          
        html.Div(className='row',
                 children=[
                       html.Div([dcc.Graph(id='barchart')],style={'width': '50%','display':'inline-block'}),
                       html.Div([dcc.Graph(id='piechart')],style={'width': '30%','display':'inline-block','height':'50%'}),  #css要并排格式调整或者要写上externalstylesheets
                       ]),
        html.Div([
            html.Div([dcc.Graph(id='scatterplot',
                      figure= px.scatter(df,x='Sales', y='Quantity', color='BigClass', hover_data=['ArtikelCode']).update_layout({'title':'Sale and Quantity',
                                                                                                                                    'showlegend':False}))],
                                                                                                                         # 'legend': {'orientation':'h',
                                                                                                                         #           'yanchor':"top",
                                                                                                                          #           'y':-0.25,
                                                                                                                          #           'xanchor':"left",
                                                                                                                           #          'x':0.01}}))],
                     style={'width': '60%','display':'inline-block'}),
            #html.Div(id='markdown',style={'width':'30%','display':'inline-block', 'textAlign':'center'})
            html.Div([dcc.Markdown(id='markdown',style={'textposition':'top center'})],style={'width': '20%','display':'inline-block','height':'100%','marginTop':0})
        ])
    ])


In [7]:
@app.callback(
        [Output('barchart', 'figure'),
        Output('piechart', 'figure')],
        [Input('dropdown', 'value'),
        Input('tabledata', 'selected_rows'),]
        )
def update_date(dropdownval,chosen_rows):
    if len(chosen_rows)==0:
        dff=df_table.groupby(dropdownval, as_index=False).sum()
    else:
        df_1=df_table[df_table.index.isin(chosen_rows)]
        dff=df_1.groupby(dropdownval, as_index=False).sum()  
    fig_bar=px.bar(
                data_frame=dff,
                x=dropdownval,
                y='Sales',
                text=dff['Sales'].apply(lambda x: round(x,2)),
                title="The total sales of {}".format(dropdownval.lower()),
            ).update_layout(title={'font':{'size':20},'xanchor':'center', 'yanchor':'top','x':0.5, 'y':0.9},
                            legend=dict(
                                x=0,
                                y=1,
                                traceorder="reversed",
                                title_font_family="Times New Roman",
                                font=dict(
                                    family="Courier",
                                    size=12,
                                    color="black"
                                ),
                                bgcolor="LightSteelBlue",
                                bordercolor="Black",
                                borderwidth=2
                                ))
    
    fig_pie=px.pie(
                data_frame=dff,
                values='Sales',
                names=dropdownval,
                title="The percentage sales of {}".format(dropdownval.lower()),
                )
    
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
                 
    fig_pie.update_layout(title={'font':{'size':20},
                                 'xanchor':'center', 
                                 'yanchor':'top','x':0.5, 'y':0.9},
                          margin={'l':0})
    
    return fig_bar,fig_pie


In [8]:
@app.callback(
        Output('markdown', 'children'),
         [Input('scatterplot', 'hoverData')])
def update_graph(hoverData):
        v_index=hoverData['points'][0]['pointIndex']
        stats = """
            **Big Category:** {big}\n
            **Middle Category:** {middle}\n
            **Fine Category:** {fine}\n
            **on date:** {date}""".format(big=df.iloc[v_index]['BigClass'],
                                    middle=df.iloc[v_index]['MiddleClass'],
                                    fine=df.iloc[v_index]['FineClass'],
                                    date=df.iloc[v_index]['DateTime'])
            
        return stats

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_2m1601801012.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_2m1601801012.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_2m1601801012.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_2m1601801012.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-component-suites/dash_table/bundle.v4_10_1m1601801011.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_12_1m1601800821.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 10:31:31] "GET /_dash-compone

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\ykkow\AppData\Roaming\Python\Python37\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ykkow\AppData\Roaming\Python\Python37\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\ykkow\AppData\Roaming\Python\Python37\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\ykkow\AppData\Roaming\Python\Python37\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\ykkow\AppData\Roaming\Python\Python37\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\ykkow\AppData\Roaming\Python\Python37\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)

127.0.0.1 - - [17/Nov/2021 10:31:33] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [17/Nov/2021 10:31:35] "POST /_dash-update-component HTTP/1.1" 200 -
